In [1]:
# CIARAN QUINLAN - CA1-Storage Solutions for Big Data - 5TH NOV 2023
### Programme Title/Year:	Higher Diploma in Data Analytics for Business 
### Module Title(s):	Storage Solutions for Big Data
### Lecturer Name(s)	Dr. Muhammad Iqbal
#### Student Full Name:	Ciaran Quinlan, Student Number: sbs23098, sbs23098@student.cct.ie

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
# sc master - running locally
sc.master

'local[*]'

In [4]:
# Import regex module and other needed libraries
import re
from operator import add
import pandas as pd #pandas lib for spark to pd
from pyspark.sql import SparkSession  # needed for a sparksession


In [5]:
# TESTING CODE Read input file from hadoop directory
# Copy a file from the hadoop to do some reading of the text.
#irishnames = sc.textFile('/ca/irishnames.csv')
#usanames = sc.textFile('/ca/usa_states_baby_names_count_and_rank.csv')
# Count lines in irish names & usa Names
#print('number of lines in irish file: %s' % irishnames.count())
#print('number of lines in usa file: %s' % usanames.count())
# if this is working the it shows my file can be read from the hadoop, the usa file take a while

In [6]:
# now read both datasets it into a spark dataframe

In [7]:
# Initialize a SparkSession
spark = SparkSession.builder \
    .appName("Read CSV from HDFS") \
    .getOrCreate()

# Read the ire CSV file from hadoop HDFS into a sparc df
hdfs_path = "hdfs:///ca/ssbd_ca1_merged_dataset.csv"
names_spark_df = spark.read.csv(hdfs_path, header=True, inferSchema=True)

23/10/30 11:14:35 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [8]:
# Show the Spark DataFrames
names_spark_df.head()

Row(Year=1910, Name='Dorothy', Sex='F', Count=57.0, IRE-Rank=0.0, USA-State-Total=1373, Country='USA', State='OR')

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower
import re

def to_sentence_case(text):
    if not text:
        return text
    return text[0].upper() + text[1:].lower()
# Read the merged CSV file from hadoop HDFS into a sparc df
#hdfs_path = "hdfs:///ca/ssbd_ca1_merged_dataset.csv"
#names_spark_df = spark.read.csv(hdfs_path, header=True, inferSchema=True)

nametofind = input("Please enter a Name: ")
# Convert input to sentence case
nametofind = to_sentence_case(nametofind)
print("You entered:", nametofind)
# Escape any special characters in the user input
nametofind = re.escape(nametofind)

# Construct the regex pattern
pattern = f".*\\b{nametofind}\\b.*"

# Filter the DataFrame based on the pattern
names_occurances_spark_df = names_spark_df.filter(col('Name').rlike(pattern))

# Show the filtered DataFrame
names_occurances_spark_df.show(20, False)

Please enter a Name: tom
You entered: Tom
+----+----+---+-----+--------+---------------+-------+-----+
|Year|Name|Sex|Count|IRE-Rank|USA-State-Total|Country|State|
+----+----+---+-----+--------+---------------+-------+-----+
|1911|Tom |M  |6.0  |0.0     |753            |USA    |OR   |
|1914|Tom |M  |5.0  |0.0     |3180           |USA    |OR   |
|1916|Tom |M  |8.0  |0.0     |4435           |USA    |OR   |
|1917|Tom |M  |6.0  |0.0     |4547           |USA    |OR   |
|1918|Tom |M  |8.0  |0.0     |5035           |USA    |OR   |
|1920|Tom |M  |8.0  |0.0     |5622           |USA    |OR   |
|1921|Tom |M  |12.0 |0.0     |6005           |USA    |OR   |
|1922|Tom |M  |9.0  |0.0     |5733           |USA    |OR   |
|1923|Tom |M  |9.0  |0.0     |5696           |USA    |OR   |
|1924|Tom |M  |12.0 |0.0     |6003           |USA    |OR   |
|1925|Tom |M  |7.0  |0.0     |5970           |USA    |OR   |
|1926|Tom |M  |12.0 |0.0     |5812           |USA    |OR   |
|1927|Tom |M  |6.0  |0.0     |5828         

In [16]:

../ssbd_ca1_merged_dataset.csv



names_occurances_spark_df.describe()

DataFrame[summary: string, Year: string, Name: string, Sex: string, Count: string, IRE-Rank: string, USA-State-Total: string, Country: string, State: string]

In [17]:
df = names_occurances_spark_df.toPandas()
df.head


<bound method NDFrame.head of       Year Name Sex  Count  IRE-Rank  USA-State-Total  Country State
0     1911  Tom   M    6.0       0.0              753      USA    OR
1     1914  Tom   M    5.0       0.0             3180      USA    OR
2     1916  Tom   M    8.0       0.0             4435      USA    OR
3     1917  Tom   M    6.0       0.0             4547      USA    OR
4     1918  Tom   M    8.0       0.0             5035      USA    OR
...    ...  ...  ..    ...       ...              ...      ...   ...
3041  2017  Tom   M  155.0      50.0                0  Ireland   IRE
3042  2018  Tom   M  198.0      32.0                0  Ireland   IRE
3043  2019  Tom   M  163.0      38.0                0  Ireland   IRE
3044  2020  Tom   M  137.0      43.0                0  Ireland   IRE
3045  2021  Tom   M  181.0      33.0                0  Ireland   IRE

[3046 rows x 8 columns]>

In [13]:
import plotly.graph_objects as go

import pandas as pd
#df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')
#  Convert to Pandas DataFrame


df = names_occurances_spark_df.toPandas()

fig = go.Figure(data=go.Choropleth(
    locations=df['State'], # Spatial coordinates
    z = df['Count'], # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
    colorbar_title = "Millions USD",
))

fig.update_layout(
    title_text = '2011 US Agriculture Exports by State',
    geo_scope='usa', # limite map scope to USA
)

fig.show()